In [14]:
import pandas as pd
weather_set = pd.read_csv("CSV/Weather_losAngeles.csv", sep=",", index_col="timestamp")

In [15]:
weather_set.head()

,city,temperature,cloud_cover,cloud_cover_description,pressure,windspeed,precipitation,felt_temperature
timestamp,,,,,,,,
2015-01-01 08:53:00,Burbank,3.0,33.0,Fair,989.11,0.0,0.0,3.0
2015-01-01 09:53:00,Burbank,2.0,33.0,Fair,989.77,0.0,0.0,2.0
2015-01-01 10:53:00,Burbank,2.0,33.0,Fair,989.44,0.0,0.0,2.0
2015-01-01 11:53:00,Burbank,1.0,33.0,Fair,989.77,0.0,0.0,1.0
2015-01-01 12:53:00,Burbank,1.0,33.0,Fair,990.10,0.0,0.0,1.0


In [16]:
weather_set.columns

Index(['city', 'temperature', 'cloud_cover', 'cloud_cover_description',
       'pressure', 'windspeed', 'precipitation', 'felt_temperature'],
      dtype='object')

In [17]:
weather_set.describe()

,temperature,cloud_cover,pressure,windspeed,precipitation,felt_temperature
count,77260.000000,77281.000000,77335.000000,77151.000000,77362.000000,77254.000000
mean,18.405177,30.304214,986.871789,8.452165,0.044915,18.249579
std,6.564116,5.243314,3.650721,6.813668,0.398354,6.418661
min,-17.000000,4.000000,968.370000,0.000000,0.000000,-24.000000
25%,14.000000,27.000000,984.500000,0.000000,0.000000,14.000000
50%,18.000000,33.000000,986.480000,7.000000,0.000000,18.000000
75%,22.000000,34.000000,989.110000,13.000000,0.000000,22.000000
max,54.000000,47.000000,1002.940000,65.000000,18.540000,44.000000


In [18]:
weather_set.info()

<class 'pandas.core.frame.DataFrame'>
Index: 77362 entries, 2015-01-01 08:53:00 to 2023-01-01 07:55:00
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   city                     77362 non-null  object 
 1   temperature              77260 non-null  float64
 2   cloud_cover              77281 non-null  float64
 3   cloud_cover_description  77281 non-null  object 
 4   pressure                 77335 non-null  float64
 5   windspeed                77151 non-null  float64
 6   precipitation            77362 non-null  float64
 7   felt_temperature         77254 non-null  float64
dtypes: float64(6), object(2)
memory usage: 5.3+ MB


In [19]:
weather_set.isnull()

,city,temperature,cloud_cover,cloud_cover_description,pressure,windspeed,precipitation,felt_temperature
timestamp,,,,,,,,
2015-01-01 08:53:00,False,False,False,False,False,False,False,False
2015-01-01 09:53:00,False,False,False,False,False,False,False,False
2015-01-01 10:53:00,False,False,False,False,False,False,False,False
2015-01-01 11:53:00,False,False,False,False,False,False,False,False
2015-01-01 12:53:00,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
2023-01-01 06:36:00,False,False,False,False,False,False,False,False
2023-01-01 06:53:00,False,False,False,False,False,False,False,False
2023-01-01 07:18:00,False,False,False,False,False,False,False,False


In [20]:
weather_set.dropna(axis=0, inplace=False)

,city,temperature,cloud_cover,cloud_cover_description,pressure,windspeed,precipitation,felt_temperature
timestamp,,,,,,,,
2015-01-01 08:53:00,Burbank,3.0,33.0,Fair,989.11,0.0,0.00,3.0
2015-01-01 09:53:00,Burbank,2.0,33.0,Fair,989.77,0.0,0.00,2.0
2015-01-01 10:53:00,Burbank,2.0,33.0,Fair,989.44,0.0,0.00,2.0
2015-01-01 11:53:00,Burbank,1.0,33.0,Fair,989.77,0.0,0.00,1.0
2015-01-01 12:53:00,Burbank,1.0,33.0,Fair,990.10,0.0,0.00,1.0
...,...,...,...,...,...,...,...,...
2023-01-01 06:36:00,Burbank,13.0,12.0,Rain,979.24,30.0,2.03,13.0
2023-01-01 06:53:00,Burbank,13.0,12.0,Rain / Windy,978.91,33.0,2.79,13.0
2023-01-01 07:18:00,Burbank,13.0,40.0,Heavy Rain,978.25,31.0,1.27,13.0


In [21]:
len(weather_set) - len(weather_set.dropna())

416

In [22]:
clean_weather_set = weather_set.dropna(axis=0)
clean_weather_set

,city,temperature,cloud_cover,cloud_cover_description,pressure,windspeed,precipitation,felt_temperature
timestamp,,,,,,,,
2015-01-01 08:53:00,Burbank,3.0,33.0,Fair,989.11,0.0,0.00,3.0
2015-01-01 09:53:00,Burbank,2.0,33.0,Fair,989.77,0.0,0.00,2.0
2015-01-01 10:53:00,Burbank,2.0,33.0,Fair,989.44,0.0,0.00,2.0
2015-01-01 11:53:00,Burbank,1.0,33.0,Fair,989.77,0.0,0.00,1.0
2015-01-01 12:53:00,Burbank,1.0,33.0,Fair,990.10,0.0,0.00,1.0
...,...,...,...,...,...,...,...,...
2023-01-01 06:36:00,Burbank,13.0,12.0,Rain,979.24,30.0,2.03,13.0
2023-01-01 06:53:00,Burbank,13.0,12.0,Rain / Windy,978.91,33.0,2.79,13.0
2023-01-01 07:18:00,Burbank,13.0,40.0,Heavy Rain,978.25,31.0,1.27,13.0


In [23]:
print("weather_set:",len(weather_set))
print("clean_weather_set:",len(clean_weather_set))
print("Datasets with null values:",len(weather_set) - len(weather_set.dropna()))

weather_set: 77362
clean_weather_set: 76946
Datasets with null values: 416


In [24]:
weather_set[weather_set.isnull().any(axis=1)]

,city,temperature,cloud_cover,cloud_cover_description,pressure,windspeed,precipitation,felt_temperature
timestamp,,,,,,,,
2015-01-26 17:53:00,Burbank,NaN,28.0,Mostly Cloudy,988.78,9.0,0.00,NaN
2015-05-27 17:51:00,Burbank,17.0,NaN,NaN,987.80,7.0,0.00,17.0
2015-05-27 17:53:00,Burbank,17.0,NaN,NaN,987.80,11.0,0.00,17.0
2015-05-27 18:53:00,Burbank,19.0,34.0,Fair,987.47,NaN,0.00,19.0
2015-08-03 16:53:00,Burbank,25.0,NaN,NaN,985.16,6.0,0.00,26.0
...,...,...,...,...,...,...,...,...
2022-11-08 12:53:00,Burbank,NaN,40.0,Heavy Rain,982.86,13.0,4.83,NaN
2022-11-16 12:53:00,Burbank,18.0,33.0,Fair,993.72,NaN,0.00,18.0
2022-11-20 21:53:00,Burbank,23.0,34.0,Fair,991.09,NaN,0.00,23.0


In [25]:
metro_set = pd.read_csv("CSV/metro_2017_2022.csv", sep=",", index_col="transaction_id")

In [26]:
metro_set.columns

Index(['start_time', 'end_time', 'start_station_id', 'start_station_lat',
       'start_station_lon', 'end_station_id', 'end_station_lat',
       'end_station_lon', 'bike_id'],
      dtype='object')

In [27]:
metro_set.describe()

,start_station_id,start_station_lat,start_station_lon,end_station_id,end_station_lat,end_station_lon
count,1.552782e+06,1.542572e+06,1.542572e+06,1.552782e+06,1.513071e+06,1.513070e+06
mean,3.615318e+03,3.404273e+01,-1.182708e+02,3.602597e+03,3.404177e+01,-1.182709e+02
std,6.527251e+02,3.053914e-01,2.180630e+00,6.507752e+02,4.389338e-01,2.166677e+00
min,3.000000e+03,3.371098e+01,-1.184954e+02,3.000000e+03,-9.000000e+01,-1.184954e+02
25%,3.032000e+03,3.402765e+01,-1.183056e+02,3.031000e+03,3.402762e+01,-1.183056e+02
50%,3.074000e+03,3.404613e+01,-1.182587e+02,3.069000e+03,3.404607e+01,-1.182587e+02
75%,4.250000e+03,3.405194e+01,-1.182483e+02,4.248000e+03,3.405110e+01,-1.182483e+02
max,4.638000e+03,5.570553e+01,1.182383e+02,4.640000e+03,5.570553e+01,3.760654e+01


In [28]:
metro_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1552782 entries, 0 to 1552781
Data columns (total 9 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   start_time         1552782 non-null  object 
 1   end_time           1552782 non-null  object 
 2   start_station_id   1552782 non-null  int64  
 3   start_station_lat  1542572 non-null  float64
 4   start_station_lon  1542572 non-null  float64
 5   end_station_id     1552782 non-null  int64  
 6   end_station_lat    1513071 non-null  float64
 7   end_station_lon    1513070 non-null  float64
 8   bike_id            1552782 non-null  object 
dtypes: float64(4), int64(2), object(3)
memory usage: 118.5+ MB


In [29]:
clean_metro_set = metro_set.dropna(axis=0)
clean_metro_set

,start_time,end_time,start_station_id,start_station_lat,start_station_lon,end_station_id,end_station_lat,end_station_lon,bike_id
transaction_id,,,,,,,,,
0,16/45/00 29-05-2018,17/08/00 29-05-2018,3068,34.053200,-118.250954,3014,34.056610,-118.237213,5772
1,18/34/00 09-08-2018,19/09/00 09-08-2018,4126,33.710979,-118.284668,4126,33.710979,-118.284668,12003
2,13/37/00 01-09-2021,13/48/00 01-09-2021,3082,34.046520,-118.237411,3031,34.044701,-118.252441,6107
3,12/40/00 31-08-2018,13/35/00 31-08-2018,4215,34.014309,-118.491341,4210,33.984341,-118.471550,6474
4,16/41/00 29-03-2018,17/15/00 29-03-2018,4126,33.710979,-118.284668,4126,33.710979,-118.284668,6374
...,...,...,...,...,...,...,...,...,...
1552776,13/32/00 30-06-2018,13/54/00 30-06-2018,3016,34.052898,-118.241562,3035,34.048401,-118.260948,6705
1552778,13/34/00 26-10-2018,13/39/00 26-10-2018,3038,34.046822,-118.248352,3030,34.051941,-118.243530,12334
1552779,13/38/00 04-10-2018,15/51/00 04-10-2018,4212,33.988129,-118.471741,4202,33.991161,-118.468292,6276


In [30]:
print("metro_set:",len(metro_set))
print("clean_metro_set:",len(clean_metro_set))
print("Datasets with null values:",len(metro_set) - len(metro_set.dropna()))

metro_set: 1552782
clean_metro_set: 1507366
Datasets with null values: 45416


In [31]:
metro_set[metro_set.isnull().any(axis=1)]

,start_time,end_time,start_station_id,start_station_lat,start_station_lon,end_station_id,end_station_lat,end_station_lon,bike_id
transaction_id,,,,,,,,,
19,22/19/00 12-07-2020,22/49/00 12-07-2020,4285,NaN,NaN,4285,NaN,NaN,15860
48,17/17/00 19-01-2020,17/23/00 19-01-2020,4285,NaN,NaN,4352,33.988419,-118.451630,15607
95,17/16/15 13-10-2019,17/23/01 13-10-2019,4285,NaN,NaN,4346,33.984341,-118.471550,15392
115,18/48/00 28-05-2020,19/59/00 28-05-2020,4285,NaN,NaN,4346,33.984341,-118.471550,15135
189,18/50/00 24-08-2019,19/37/00 24-08-2019,4286,NaN,NaN,4285,NaN,NaN,15289
...,...,...,...,...,...,...,...,...,...
1552671,18/03/00 12-07-2021,18/04/00 12-07-2021,4567,34.020180,-118.404030,3000,NaN,NaN,15544
1552703,13/51/00 15-05-2019,14/04/00 15-05-2019,4286,NaN,NaN,4286,NaN,NaN,16178
1552704,18/46/00 28-10-2022,18/51/00 28-10-2022,4450,34.063492,-118.287163,3000,NaN,NaN,15520
